# Assignment 3

### 1.复习上课内容

### 2.回答以下理论问题

#### 1. 请写一下TF-IDF的计算公式
 TFt,d = count(t,d)
 IDFt = log10(N / DFt)
 TF-IDF = TFt,d * IDFt

#### 2. LDA算法的基本假设是什么？
1.原始数据根据样本均值进行分类。
2.不同类的数据拥有相同的协方差矩阵。

#### 3. 在TextRank算法中构建图的权重是如何得到的？
权重是从某个点链接出去的数量的倒数，数量越多，权重越小。
两个词之间的权重是这两个词向量的余弦距离。

#### 4. 什么是命名实体识别？ 有什么应用场景？
命名实体识别是指识别文本中具有特定意义的实体，包括人名、地名、组织名、动物名、植物名等等。识别出这些实体的名称，并且识别出这些实体的类别。
命名实体识别可以用于知识图谱、文本理解、对话系统、舆情分析等等。

#### 5.NLP主要有哪几类任务 ？
分类任务：文本分类，情感分类。
生成任务：机器翻译，对话系统，文本摘要

### 3.实践题

#### 3.1 手动实现TextRank算法 (在新闻数据中随机提取1条新闻训练词向量和做做法测试）

 提示：
 1. 确定窗口，建立图链接。   
 2. 根据公式实现算法迭代(d=0.85)

In [23]:
from gensim import corpora, models
import jieba.posseg as jp
import jieba
import pandas as pd
import numpy as np

In [64]:
# 加载数据，选取前三条新闻
data_source = "/Users/zhangxiaonan/Desktop/article_9k.txt"
data = pd.read_csv(data_source, header=None)
data = data.fillna('')
news_content = data[0].tolist()
print(news_content[:3])


['此外自本周6月12日起除小米手机6等15款机型外其余机型已暂停更新发布含开发版体验版内测稳定版暂不受影响以确保工程师可以集中全部精力进行系统优化工作有人猜测这也是将精力主要用到MIUI9的研发之中MIUI8去年5月发布距今已有一年有余也是时候更新换代了当然关于MIUI9的确切信息我们还是等待官方消息', '骁龙835作为唯一通过Windows10桌面平台认证的ARM处理器高通强调不会因为只考虑性能而去屏蔽掉小核心相反他们正联手微软找到一种适合桌面平台的兼顾性能和功耗的完美方案报道称微软已经拿到了一些新的源码以便Windows10更好地理解biglittle架构资料显示骁龙835作为一款集成了CPUGPU基带蓝牙WiFi的SoC比传统的Wintel方案可以节省至少30的PCB空间按计划今年Q4华硕惠普联想将首发骁龙835Win10电脑预计均是二合一形态的产品当然高通骁龙只是个开始未来也许还能见到三星Exynos联发科华为麒麟小米澎湃等进入Windows10桌面平台', '此前的一加3T搭载的是3400mAh电池DashCharge快充规格为5V4A至于电池缩水可能与刘作虎所说一加手机5要做市面最轻薄大屏旗舰的设定有关按照目前掌握的资料一加手机5拥有55寸1080P三星AMOLED显示屏6G8GBRAM64GB128GBROM双1600万摄像头备货量惊喜根据京东泄露的信息一加5起售价是xx99元应该是在279928992999中的某个']


# 基于LDA的TextRank的实现

In [86]:
def get_text(texts):
    # 保留特定词性
    flags = ('n', 'nr', 'ns', 'nt', 'eng', 'v', 'd')  # 词性
    # 去掉停用词
    stop_words = []
    with open('/Users/zhangxiaonan/Desktop/百度停用词表.txt') as f:
        for word in f.readlines():
            stop_words.append(word.strip())
            
    words_list = []
    news = ''
    for text in texts:
        words = [w.word for w in jp.cut(text) if w.flag in flags and w.word not in stop_words]
        words_list.append(words)
        
        # 分词后的整体字符串
#         if words:
#             word_list.append(words)
#     for i in word_list:
#         for j in i:
#             if i:
#                 news+=j

    # 返回分词后的词组list
    return words_list

words_list = get_text(news_content[:3])
print(words_list)

[['起除', '小米', '手机', '机型', '机型', '已', '暂停', '更新', '发布', '含', '开发', '版', '体验版', '内测', '稳定版', '暂不', '受', '确保', '工程师', '精力', '系统优化', '精力', '用到', 'MIUI9', 'MIUI8', '发布', '距今已有', 'MIUI9', '信息', '等待', '官方消息'], ['Windows10', '桌面', '平台', '认证', 'ARM', '处理器', '高通', '只', '性能', '去', '核心', '正', '联手', '找到', '适合', '桌面', '平台', '兼顾', '性能', '功耗', '方案', '报道', '称', '拿到', '源码', 'Windows10', '更好', '理解', 'biglittle', '架构', '资料', '显示', '集成', 'CPUGPU', '基带', '蓝牙', 'WiFi', 'SoC', '传统', 'Wintel', '方案', '节省', '至少', 'PCB', '空间', '按计划', 'Q4', '华硕', '惠普', '联想', '首发', 'Win10', '电脑', '均', '形态', '产品', '高通', '也许', '见到', 'Exynos', '联发科', '华为', '麒麟', '小米', '澎湃', 'Windows10', '桌面', '平台'], ['一加', 'T', '搭载', 'mAh', '电池', 'DashCharge', '充', '规格', 'V4A', '电池', '缩水', '刘作虎', '一加', '手机', '做', '市面', '最', '大屏', '旗舰', '设定', '资料', '一加', '手机', '拥有', 'P', 'AMOLED', '显示屏', 'G8GBRAM64GB128GBROM', '双', '摄像头', '备货', '量', '京东', '泄露', '信息', '一加', '售价', 'xx99']]


In [88]:
# 生成LDA模型
def LDA_model(words_list):
    # Dictionary()方法遍历所有的文本，为每个不重复的单词分配一个单独的整数ID，同时收集该单词出现次数以及相关的统计信息
    dictionary = corpora.Dictionary(words_list)
    print(dictionary,'\n')
    print('打印查看每个单词的id:')
    print(dictionary.token2id,'\n')  # 打印查看每个单词的id
 
    # 将dictionary转化为一个词袋
    # doc2bow()方法将dictionary转化为一个词袋。得到的结果corpus是一个向量的列表，向量的个数就是文档数。
    # 在每个文档向量中都包含一系列元组,元组的形式是（单词ID，词频）
    corpus = [dictionary.doc2bow(words) for words in words_list]
    print('输出每个文档的向量:')
    print(corpus)  # 输出每个文档的向量
 
    # LDA主题模型
    # num_topics -- 必须要生成的主题个数。
    # id2word    -- 必须LdaModel类要求我们之前的dictionary把id都映射成为字符串。
    # passes     -- 可选模型遍历语料库的次数。遍历的次数越多，模型越精确。但是对于非常大的语料库，遍历太多次会花费很长的时间。
    lda_model = models.ldamodel.LdaModel(corpus=corpus, num_topics=2, id2word=dictionary, passes=10)
    return lda_model

lda_model = LDA_model(words_list)


Dictionary(115 unique tokens: ['MIUI8', 'MIUI9', '体验版', '信息', '内测']...) 

打印查看每个单词的id:
{'MIUI8': 0, 'MIUI9': 1, '体验版': 2, '信息': 3, '内测': 4, '发布': 5, '受': 6, '含': 7, '官方消息': 8, '小米': 9, '工程师': 10, '已': 11, '开发': 12, '手机': 13, '暂不': 14, '暂停': 15, '更新': 16, '机型': 17, '版': 18, '用到': 19, '确保': 20, '稳定版': 21, '等待': 22, '精力': 23, '系统优化': 24, '起除': 25, '距今已有': 26, 'ARM': 27, 'CPUGPU': 28, 'Exynos': 29, 'PCB': 30, 'Q4': 31, 'SoC': 32, 'WiFi': 33, 'Win10': 34, 'Windows10': 35, 'Wintel': 36, 'biglittle': 37, '也许': 38, '产品': 39, '传统': 40, '兼顾': 41, '功耗': 42, '华为': 43, '华硕': 44, '去': 45, '只': 46, '均': 47, '基带': 48, '处理器': 49, '平台': 50, '形态': 51, '性能': 52, '惠普': 53, '找到': 54, '报道': 55, '拿到': 56, '按计划': 57, '方案': 58, '显示': 59, '更好': 60, '架构': 61, '核心': 62, '桌面': 63, '正': 64, '源码': 65, '澎湃': 66, '理解': 67, '电脑': 68, '称': 69, '空间': 70, '联发科': 71, '联想': 72, '联手': 73, '至少': 74, '节省': 75, '蓝牙': 76, '见到': 77, '认证': 78, '资料': 79, '适合': 80, '集成': 81, '首发': 82, '高通': 83, '麒麟': 84, 'AMOLED': 85, 'DashCharge': 8

In [91]:
# 用 print_topic 和 print_topics 方法来查看主题
topic_words = lda_model.print_topics(num_topics=2, num_words=5)
print('打印所有topic，每个topic显示5个词:')
print(topic_words,'\n') 

words_list = lda_model.show_topic(0, 5)
print('输出topic的的词及其词的权重:')
print(words_list)

打印所有topic，每个topic显示5个词:
[(0, '0.028*"精力" + 0.028*"机型" + 0.028*"MIUI9" + 0.028*"发布" + 0.017*"信息"'), (1, '0.028*"一加" + 0.021*"Windows10" + 0.021*"平台" + 0.021*"桌面" + 0.015*"手机"')] 

输出topic的的词及其词的权重:
[('精力', 0.02811364), ('机型', 0.028096432), ('MIUI9', 0.028096361), ('发布', 0.028062385), ('信息', 0.01694779)]


In [93]:
# 选取第七条新闻
news = data[0][6]
#print(data[0][6])
words_ls = []
for text in news:
    words = [w.word for w in jp.cut(text) if w.flag in flags and w.word not in stop_words]
    words_ls.append(words)

words = [[w.word for w in jp.cut(news) if w.flag in flags and w.word not in stop_words]]
dictionary = corpora.Dictionary(words_ls)
text_corpus = [dictionary.doc2bow(word) for word in words]
list(lda_model[text_corpus])
#结果表示预测可能是第二个topic

[[(0, 0.25918934), (1, 0.74081063)]]

#### 选做 1.  提取新闻人物里的对话。(使用以上提取小数据即可）

提示：    
1.寻找预料里具有表示说的意思。    
2.使用语法分析提取句子结构。    
3.检测谓语是否有表示说的意思。(手工设定，比如"说，表达，表示")